<img src="images/header.png" alt="Logo UCLM-ESII" align="right">

<br><br><br><br>
<h2><font color="#92002A" size=4>Trabajo Fin de Grado</font></h2>

<h1><font color="#6B001F" size=5>Generación automática de playlist de canciones <br> mediante técnicas de minería de datos</font></h1>
<h2><font color="#92002A" size=3>Parte 3 - Descarga de playlists</font></h2>

<br>
<div style="text-align: right">
    <font color="#B20033" size=3><strong>Autor</strong>: <em>Miguel Ángel Cantero Víllora</em></font><br>
    <br>
    <font color="#B20033" size=3><strong>Directores</strong>: <em>José Antonio Gámez Martín</em></font><br>
    <font color="#B20033" size=3><em>Juan Ángel Aledo Sánchez</em></font><br>
    <br>
<font color="#B20033" size=3>Grado en Ingeniería Informática</font><br>
<font color="#B20033" size=2>Escuela Superior de Ingeniería Informática | Universidad de Castilla-La Mancha</font>

</div>

---

<br>

<a id="indice"></a>
<h2><font color="#92002A" size=5>Índice</font></h2>

<br>

* [1. Introducción](#section1)
* [2. Descarga de playlists](#section2)
    * [2.1. Métodos auxiliares](#section21)
    * [2.2. Métodos pertenecientes al proceso de descarga](#section22)
* [3. Proceso de descarga](#section3)

<br>

---

In [ ]:
# Permite establecer la anchura de la celda
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
import datetime
import json
import os
import requests
import spotipy
import time
import zipfile

from spotipy.oauth2 import SpotifyClientCredentials
from tqdm.notebook import tqdm as tqdm_nb

---

<br>

<a id="section1"></a>
## <font color="#92002A">1 - Introducción</font>

<br>

En la libreta anterior, [_Preparación de datos para el proceso de descarga_](02-PreparacionDescarga.ipynb), hemos seleccionado un conjunto de identificadores de playlists para descargar. Este conjunto es bastante grande, puesto que al no disponer de información suficiente con los resultados que nos ha devuelto el proceso de búsqueda (identificador, título y número de pistas que contiene la lista de reproducción), necesitamos descargarlos para decidir cuáles de ellos seleccionamos para nuestro DataSet.


Para facilitar la tarea de descarga, vamos a establecer 2 de los criterios que habíamos indicado en la libreta [_Estudio del 'Million Playlist Dataset' (MPD)_](00-EstudioMPD.ipynb):

* La playlist debe tener al menos _1 seguidor_.
* En el momento de descarga, la playlist debe ser _pública_.
* La playlist no contiene pistas locales (aquellas que están almacenadas en el dispositivo del usuario y no se encuentran en *Spotify*).


Gracias a esto reduciremos el tiempo de descarga, puesto que, como primero hacemos una llamada a la API para ver si la playlist es publica y tiene más de 1 seguidor, la respuesta es inmediata (ya que el sistema no nos tiene que devolver la lista completa).

Otra ventaja que obtenemos es la del espacio de almacenamiento, ya que no almacenamos las playlists que no cumplen los criterios anteriores.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section2"></a>
## <font color="#92002A">2 - Descarga de playlists</font>

<br>

Definimos los directorios con los que vamos a trabajar junto a los campos que nos interesa obtener de las playlists:

In [ ]:
# Variables globales

# Directorio donde se encuentran los conjuntos de identificadores
# que queremos descargar (almacenados en varios ficheros .txt)
PLS_ID_PATH = 'pls_id_sets'

# Directorio que empleamos para almacenar temporalmente las playlists
TEMP_PATH = 'pls_files_tmp'

# Directorio donde almacenamos los conjuntos de playlists que hemos
# descargado
PLS_SET_PATH = 'pls_sets'

# Campos que nos interesa descargar de las playlists
FIELDS_PL ='collaborative,followers,id,name,tracks'

<br>


Creamos el gestor de _Spotipy_ que emplearemos para descargar las playlists:

In [ ]:
sp_client_id = '' #SpotifyClientID
sp_client_secret = '' #SpotifyClientSecret

# Crea el gestor
client_credentials_manager = SpotifyClientCredentials(client_id=sp_client_id, client_secret=sp_client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section21"></a>
### <font color="#B20033">2.1 - Métodos auxiliares</font>

<br>

<a id="section211"></a>
#### <font color="#B20033">Bot de Telegram</font>

<br>

Al igual que en la libreta [_Búsqueda de playlists_](01-BusquedaPlaylists.ipynb), creamos un Bot de _Telegram_ para seguir el proceso de descarga:

In [ ]:
tl_bot_token = ''
tl_bot_chatID = ''

def telegram_bot_sendtext(bot_message, prefix=''):
    """
    :param bot_message: Mensaje que publicará el bot.
    :param prefix: Prefijo con que se identificará el mensaje (opcional).
    :return: Resultado de la petición.
    """
    bot_token = tl_bot_token
    bot_chatID = tl_bot_chatID
    
    if prefix != '':
        bot_message = '[{}]: {}'.format(prefix,bot_message)
        
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&text=' + bot_message

    response = requests.get(send_text)

    return response.json()['ok']

<br>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: El argumento `prefix` lo utilizaremos para distinguir desde dónde se ha enviado el mensaje, en caso de que se empleen varios equipos o procesos para la descarga de playlists.
</div>

<br>

Para comprobar que el Bot funciona correctamente:

In [ ]:
#test = telegram_bot_sendtext('Testing Telegram bot')
#print(test)

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section212"></a>
#### <font color="#B20033">Creación de logs</font>

<br>

Para almacenar la información del proceso de búsqueda, hemos creado una función que nos permitirá guardar aquellos eventos que consideremos de interés junto a la fecha y hora en la que han ocurrido. El nombre del fichero, por defecto, es `log.txt`.

In [ ]:
def write_to_log(text,file_name='log.txt'):
    """
    :param text: Texto a guardar en el log
    :param file_name: Nombre del fichero (opcional).
    """
    current_date = datetime.datetime.now()
    with open(file_name, 'a') as fp:
        fp.write(f'({current_date}):\t{text}\n')

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section213"></a>
#### <font color="#B20033">Extracción de datos</font>

<br>

En determinados casos, si el número de pistas que contiene una playlist es demasiado grande, la respuesta a la llamada de la WebAPI no devuelve la lista completa de pistas que conforman dicha playlist. Para resolver este problema, hemos creado una función llamada ***get_playlist_tracks*** que se encargara de proporcionarnos todas las pistas que conforman la playlist solicitada.

*Fuente*: [Spotipy: How to read more than 100 tracks from a playlist (Stack Overflow)](https://stackoverflow.com/questions/39086287/spotipy-how-to-read-more-than-100-tracks-from-a-playlist).

In [ ]:
# Obtiene las pistas pertenecientes a una playlist
def get_playlist_tracks(username,pl_id):
    """
    :param username: Nombre de usuario al que pertenece la playlist.
    :pl_id: Identificador de la playlist. 
    :return: Lista con el conjunto de pistas.
    """
    results = sp.user_playlist_tracks(username,pl_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

<br>

Cuando la llamada a la WebAPI nos devuelve una playlist, para cada pista podemos ver en qué fecha se agregó a la lista. Dicha fecha, está en un formato que no es estándar. Para resolver esto, hemos creado la siguiente función que se encarga de convertir el formato de fecha de _Spotify_ a `TimeStamp`:

In [ ]:
# Convierte el formato de fecha de Spotify a TimeStamp
def spdate_to_timestamp(date):
    """
    :param date: Fecha en el formato de Spotify.
    :return: Fecha en formato 'timestamp'.
    """
    dt = datetime.datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ')
    return int(dt.timestamp())

<br>

Como no nos interesa toda la información que contiene la lista de pistas que conforman la playlist, con el método ***clean_tracks_info*** nos encargamos de quedarnos con aquellos datos que son relevantes:

* **pos**: Posición de una pista dentro de la playlist.
* **added_at**: Fecha en la que se ha agregado la pista a la playlist.
* **track_name**: Nombre de la pista.
* **track_uri**: Dirección Spotify de la pista.
* **duration_ms**: Duración en milisegundos de la pista.
* **artist_name**: Nombre del artista al que pertenece la pista. En caso de que una pista tenga más de un artista, nos quedamos sólo con el primero (artista principal).
* **artist_uri**: Dirección Spotify del artista. Al igual que en *artist_name*, en caso de que una pista tenga más de un artista, nos quedamos sólo con dirección de éste (artista principal).
* **album_name**: Nombre del álbum al que pertenece la pista.
* **album_uri**: Dirección Spotify del álbum.



In [ ]:
# Obtiene aquellos datos que nos interesan de la lista
# de pistas que contiene la playlist y devuelve una 
# lista de pistas en formato de diccionario
def clean_tracks_info(track_list):
    """
    :param track_list: Lista con el conjunto de listas a procesar.
    :return: Lista con el conjunto de pistas procesadas.
    """
    tracks = []
    for track in track_list:
        try:
            track_data = dict()
            track_data['pos'] = len(tracks)
            track_data['added_at'] = spdate_to_timestamp(track['added_at'])
            track_data['track_name'] = track['track']['name']
            track_data['track_uri'] = track['track']['uri']
            track_data['duration_ms'] = track['track']['duration_ms']
            track_data['artist_name'] = track['track']['artists'][0]['name']
            track_data['artist_uri'] = track['track']['artists'][0]['uri']
            track_data['album_name'] = track['track']['album']['name']
            track_data['album_uri'] = track['track']['album']['uri']
            tracks.append(track_data)
        except:
            # Existen ocasiones en las que una pista no esta disponible
            # Continuamos
            pass
    return tracks

<br>

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i> __Nota__: Hemos agregado un bloque _try-except_ puesto que existen ocasiones en las que una pista no contiene toda la información (ya sea porque la pista ya no se encuentra disponible en _Spotify_ u otros motivos). Si se da dicho caso, no la guardamos en la lista.
</div>

<br>

Por último, con el método ***read_file_ids*** nos encargamos de leer los identificadores de playlists que hay almacenados dentro de un fichero:

In [ ]:
# Lee los ficheros con los identificadores de
# las playlists
def read_file_ids(file_path):
    """
    :param file_path: Fichero que contiene el conjunto de identificadores. 
    :return: Lista con el conjunto de identificadores.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        ids = [line.rstrip() for line in f]
    return ids

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section214"></a>
#### <font color="#B20033">Comprobaciones</font>

<br>

Cuando buscamos una playlist que ya no se encuentra en el sistema, o que por cualquier motivo no se ha introducido de forma correcta, se produce una excepción llamada `spotipy.client.SpotifyException`. Para almacenar la información de dicha excepción, hemos creado un método llamado ***get_spotify_error*** para transformarla en un diccionario:

In [ ]:
def get_spotify_error(e):
    """
    :param e: Texto con el error devuelto por el cliente de Spotipy. 
    :return: Diccionario con la información procesada del error.
    """
    val1 = str(e).split(',')[0].split(':')[-1].strip()
    val2 = str(e).split(',')[1].split('-')[-1].split(':')[-1].strip()
    return {'status' : val1, 'message' : val2}

<br>

Como hemos indicado en la introducción, vamos a comprobar que la playlist cumple dos criterios antes de descargarla por completo:
* La playlist tiene al menos _1 seguidor_.
* En el momento de la descarga, la playlist es _pública_.

También comprobaremos que la playlist esté disponible, para evitar que se produzcan excepciones en el momento de la descarga.

Estas comprobaciones las realizaremos en la función ***pl_check_requirements***:

In [ ]:
# Comprobamos que la lista es pública, tiene al
# menos un seguidor y existe
def pl_check_requirements(pl_id):
    """
    :param pl_id: Identificador de la playlist. 
    :return: Resultado de la comprobación (bool).
    """
    try:
        pl = sp.user_playlist('',pl_id,fields='public,followers')
        if pl['public']:
            if str(pl['followers']['total']).isnumeric():
                return int(pl['followers']['total']) > 0
            else:
                return False
        else:
            return False
    except spotipy.client.SpotifyException as e:
        error = get_spotify_error(e)
        write_to_log(f'Playlist [{pl_id}]: {error}')
        return False

<br>

Otro de los requisitos que hemos establecido, es que la playlist no contenga pistas locales (aquellas que están en el dispositivo del usuario y no se encuentran en Spotify). Este criterio nos encargamos de comprobarlo con la siguiente función:

In [ ]:
# Comprueba que una lista de pistas no contiene
# ninguna que esté almacenada de forma local
def pl_has_local_files(track_list):
    """
    :param track_list: Lista con el conjunto de pistas. 
    :return: Resultado de la comprobación de pistas locales (bool).
    """
    local_list = [track['is_local'] for track in track_list]
    return any(local_list)

<br>

La última comprobación que vamos a realizar es si, para alguna pista de la playlist, el contenido de *artist_name*, *album_name* o *track_name* está ___vacío___. Existen casos en el que la pista que hay agregada a una playlist, ya no está disponible.

In [ ]:
def pl_has_incomplete_content(pl):
    """
    :param pl: Playlist a comprobar. 
    :return: Resultado de la comprobación de contenido incompleto (bool).
    """
    incomplete = False
    
    for track in pl['tracks']:
        if track['artist_name'] == '':
            incomplete = True
            break
        elif track['album_name'] == '':
            incomplete = True
            break
        elif track['track_name'] == '':
            incomplete = True
            break
        else:
            pass
        
    return incomplete

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section22"></a>
### <font color="#B20033">2.2- Métodos pertenecientes al proceso de descarga</font>

<br>

En este apartado, vamos a explicar cómo se realiza la descarga de playlists cuyos identificadores tenemos almacenados en varios ficheros de texto.

Primero, creamos una función para descargar una lista de reproducción mediante su identificador (*id*):

In [ ]:
# Descarga la playlist con id 'pl_id' de Spotify
def download_playlist(pl_id,fields=FIELDS_PL):
    """
    :param pl_id: Identificador de la playlist a descargar.
    :param fields: Campos de la playlist que se desean obtener.
    :return: Playlist en formato de diccionario.
    """
    
    # Establecemos el valor del usuario a '',
    # puesto que no es necesario indicarlo
    pl = sp.user_playlist('',pl_id,fields=fields)

    # Modificamos el valor de la key 'followers' ya que
    # obtenemos un diccionario y sólo nos interesa el número
    pl['num_followers'] = pl['followers']['total']
    del pl['followers']

    # Descargamos el conjunto completo de pistas
    # (hay que hacer separado el proceso puesto que si 
    # la lista es muy larga, se bajan las primeras n pistas)
    pl['tracks'] = get_playlist_tracks('',pl_id)

    # Si hay contenido local, ignoramos la playlist
    if not pl_has_local_files(pl['tracks']):
        pl['tracks'] = clean_tracks_info(pl['tracks'])
        pl['num_tracks'] = len(pl['tracks'])
    else:
        write_to_log(f'Playlist [{pl_id}] has local content.')
        pl = {}
        
    # Si alguna pista tiene contenido incompleto,
    # ignoramos la playlist
    if pl != {}:
        is_incomplete = pl_has_incomplete_content(pl)
        
        if is_incomplete:
            write_to_log(f'Playlist [{pl_id}] has incomplete content.')
            pl = {}
    
    return pl

Una vez que hemos descargado la playlist mediante el cliente de *Spotipy*, realizamos las siguientes tareas:
1. Extraemos el número de seguidores del diccionario, ya que no nos interesa la lista de usuarios que siguen la playlist.
2. Obtenemos la lista completa de pistas que conforman la playlist. Como vimos anteriormente, si la lista tiene más de 100 elementos sólo se descargan los 100 primeros.
3. Comprobamos que la lista *no contiene pistas locales* pertenecientes al usuario.
    * En caso de que no tenga pistas locales:
        * Extraemos la información más relevante de las pistas mediante la función *clean_tracks_info*
        * Calculamos el número total de pistas y lo almacenamos con la clave 'num_tracks' dentro del diccionario que vamos a devolver.
    * En caso de que *existan pistas locales*, vaciamos el diccionario a devolver


4. Devolvemos el diccionario _pl_ que contiene la playlist que hemos indicado.

<br>

<div class="alert alert-block alert-warning">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: Devolvemos un diccionario vacío en el caso de que la playlist indicada contenga pistas locales. Así nos resultará más fácil identificar este caso cuando procesemos el fichero que contiene los identificadores a descargar.
</div>

<br>

Seguidamente, creamos un método para procesar el fichero de texto que contiene los identificadores de las playlists que nos interesa descargar:

In [ ]:
# Procesa un fichero que contiene identificadores
# de playlists para descargar
def process_file(file_path):
    """
    :param file_path: Ruta del fichero a procesar. 
    """
    pl_ids = read_file_ids(file_path)
    file_name = os.path.basename(file_path)
    
    # Lista que empleamos para almacenar aquellas playlist
    # que se han descartado
    discarded_ids = []
    
    message = 'File [{}] processing starts.'.format(file_name)
    write_to_log(message)
    
    try:
        for pl_id in pl_ids:
            # Comprobamos que es pública, tiene seguidores
            # y está disponible
            if pl_check_requirements(pl_id):
                pl = download_playlist(pl_id)
                time.sleep(0.2) # Retardo
                
                if bool(pl):
                    pl_path = '{}/{}.json'.format(TEMP_PATH,pl_id)
                    with open(pl_path, 'w') as file:
                        json.dump(pl,file)
                else:
                    # La descartamos, ya que si el diccionario devuelto
                    # esta vacío, significa que contiene pistas locales
                    discarded_ids.append(pl_id)
            else:
                discarded_ids.append(pl_id)
    except Exception as e:
        time.sleep(300)
        
        message = 'File [{}] - Something went wrong. Retrying...'.format(file_name)
        write_to_log(message)
        write_to_log(str(e))
        telegram_bot_sendtext(message)
        
        process_file(file_path)

    pls_list = []
    for pl_id in pl_ids:
        if pl_id not in discarded_ids:
            pl_path = '{}/{}.json'.format(TEMP_PATH,pl_id)
            with open(pl_path) as json_file:  
                data = json.load(json_file)
                pls_list.append(data)
            os.remove(pl_path)
  
    file_id = int(file_name.split('.')[0].split('_')[-1])
    pls_file_name = 'pls-set_{:05d}.json'.format(file_id)
    with open(os.path.join(PLS_SET_PATH,pls_file_name), 'w') as file:
        json.dump(pls_list,file,indent=4)
    
    os.remove(file_path)
    
    message = 'File [{}] processed correctly.'.format(file_name)
    write_to_log(message)

El proceso que realiza la función anterior es el siguiente:

1. Leemos los identificadores que tiene el fichero haciendo uso de la función ***read_file_ids***
2. Creamos una lista, ***discarted_ids***, donde almacenaremos aquellos identificadores que se han omitido (ya sea porque la lista es privada, no tiene seguidores o no esté disponible).
3. Para cada identificador que hayamos obtenido de la lista:
    * Comprobamos que cumpla los requisitos que hemos establecido.
        * En caso de que los cumpla, comprobamos que no tiene pistas locales y lo almacenamos en un fichero _JSON_ dentro de la carpeta temporal que hemos indicado (`TEMP_PATH`). 
        * En caso de que tenga pistas locales, lo añadimos a la lista de ids descartados.
    * Si no cumple los requisitos, añadimos el identificador a la lista de descartes y continuamos.
4. En caso de que se produzca una excepción en el proceso de descarga de playlists, esperaremos 5 minutos y volvemos a intentarlo.
5. Una vez que se han descargado todas las playlist del fichero:
    * Leemos las playlist de los ficheros _JSON_ que hemos almacenado en la carpeta temporal (comprobando que no sean aquellas que hemos descartado) y los almacenamos en una lista llamada ***pls_list***.
    * Una vez que se han leído todas las playlist, guardamos la lista que las contiene en un único fichero JSON dentro de la carpeta que hemos indicada en `PLS_SET_PATH`.
6. Borramos el fichero que contiene los identificadores de las playlist que hemos descargado.

<br>

<div class="alert alert-block alert-warning">

<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: Al igual que en el proceso de búsqueda, tras descargar una playlist hemos introducido un retardo para no saturar el servidor realizando miles de peticiones consecutivas. Aparte de evitar cualquier saturación, si no lo hiciéramos infringiríamos uno de los [_Términos del Servicio (ToS)_](https://developer.spotify.com/terms/#iv) de _Spotify_.
</div>

<br>

Por último, definimos la función que se encargará de procesar la carpeta que contiene los ficheros de texto con los identificadores de las playlist que deseamos descargar:

In [ ]:
# Procesa el directorio que contiene los
# ficheros con los identificadores de las
# playlists
def process_folder(folder_path):
    """
    :param folder_path: Ruta del directorio a procesar. 
    """
    files_list = list()
    
    # Creamos el directorio temporal en caso
    # de que no exista
    if not os.path.exists(TEMP_PATH):
        os.makedirs(TEMP_PATH)
    
    # Creamos el directorio temporal en caso
    # de que no exista
    if not os.path.exists(PLS_SET_PATH):
        os.makedirs(PLS_SET_PATH)
    
    # Almacenamos en una lista, aquellos ficheros
    # que contienen los identificadores
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(PLS_ID_PATH,file_name)
        if os.path.isfile(file_path) and 'pls-id-set_' in file_name:
            files_list.append(file_path)

    # Procesamos uno a uno los ficheros que contiene
    # la lista 'files_list'
    for file in tqdm_nb(files_list):
        process_file(file)
        time.sleep(30) # Retardo
    
    # Borramos los directorios temporal y de lectura,
    # ya que una vez terminado el proceso quedan vacíos
    os.rmdir(TEMP_PATH)
    os.rmdir(PLS_ID_PATH)
    
    message = 'Download Complete!'
    telegram_bot_sendtext(message)

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section3"></a>
## <font color="#92002A">3 - Proceso de descarga</font>
<br>

Una vez que hemos declarado todo el código necesario para la descarga de playlists, llamamos a la función ***process_folder*** pasándole como argumento la ruta dónde se encuentran los ficheros con los identificadores de las playlist (en nuestro caso `PLS_ID_PATH`) para comenzar con el proceso:

In [ ]:
process_folder(PLS_ID_PATH)

Tras descargarse y procesarse los conjuntos de playlists, comprimimos cada fichero que se ha generado para que ocupe menos espacio en el dispositivo de almacenamiento:

In [ ]:
for file_name in tqdm_nb(os.listdir(PLS_SET_PATH)):
    file_path = os.path.join(PLS_SET_PATH, file_name)

    if file_name.endswith(".json"):
        zip_file_name = os.path.splitext(file_name)[0] + ".zip"
        with zipfile.ZipFile(os.path.join(PLS_SET_PATH, zip_file_name),'w') as zip: 
             zip.write(file_path, compress_type=zipfile.ZIP_DEFLATED, arcname=file_name)
        os.remove(file_path)

Los nuevos ficheros se guardan en el mismo directorio donde se encuentran los *JSON*, los cuales serán eliminados tras comprimirse.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-graduation-cap" aria-hidden="true" style="color:#92002A"></i> </font></div>